In [1]:
import numpy as np, PIL.Image as Image, matplotlib.pyplot as plt, os, pandas as pd, datasets, torch
from utility_scripts.pose_estimation_utilities import euler_angles_to_matrix, matrix_to_euler_angles, draw_axis

In [2]:
datasets_list = [
    datasets.MS1MV2(),
    datasets.AffectNet(),
    datasets.RAFDB(),
]

In [3]:
loader, sample_weights = datasets.get_balanced_loader(datasets_list, batch_size = 16, num_workers = 2, epoch_size = 100)

tensor(1.0000, dtype=torch.float64)


In [21]:
images, labels = next(iter(loader))

In [22]:
(labels['face_recognition'] != -1).sum()

tensor(4)

In [23]:
(labels['emotion'] != -1).sum()

tensor(12)